In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import glob
import os

plt.style.use('ggplot') # Make the graphs a bit prettier

In [4]:
data = {}

for path in glob.glob(os.path.expanduser("~/dev/ma/data/results/pre_dlm0/*.json")):
  for typename, items in json.load(open(path)).items():
    if typename in data:
      data[typename].update(items)
    else:
      data[typename] = items

frames = { typename: pd.DataFrame.from_dict(items, orient='index') for typename, items in data.items() }

In [16]:
frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['graph', 'binary'])['modularity', 'runtime'] \
    .agg(['mean', 'size'])
    

modularity  \
                                                                                                            mean   
graph                                              binary                                                          
graphs/com-friendster-preprocessed-*.bin           /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.532608   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.576456   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.590089   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.606714   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.599051   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.603017   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.617201   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.623566   
graphs/mu-04/graph_50_10000_mu_0.4_10000000-pre... /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.586742   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.594388   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.588561   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.598283   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.576640   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.591265   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.596292   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.595724   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.584280   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...   0.590549   

                                                                                                            \
                                                                                                      size   
graph                                              binary                                                    
graphs/com-friendster-preprocessed-*.bin           /home/kit/iti/kp0036/code/prototypes/thrill_lou...    2   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    2   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    1   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    5   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    1   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    1   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    2   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    1   
graphs/mu-04/graph_50_10000_mu_0.4_10000000-pre... /home/kit/iti/kp0036/code/prototypes/thrill_lou...    6   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    4   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    5   
                                                   /home/kit/iti/kp0036/code/prototypes/thrill_lou...    4   
               